# Redes de Convolução Dilatadas

Nesta aula, vamos ver como usar redes de convolução para prever o tempo. Para isso, vamos usar um tipo de CNN que suporta uma operação diferente de convolução, a _convolução dilatada causal_.

In [ ]:
from __future__ import print_function, division

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Introdução

CNNs foram projetadas principalmente para processamento espacial. Elas, contudo, são usadas também como sucesso para entradas de natureza sequencial, como frases, áudio etc. Nestes casos, elas tiram proveito do contexto observado na sequência, sem lidar realmente com o seu encadeamento sequêncial. Contudo, mais recentemente, arquiteturas têm sido propostas que são capazes de lidar mesmo com a natureza sequencial da informação. Um exemplo são as CNNs dilatadas (DC-CNNs), ou seja, CNNs com operações de convolução causais dilatadas.

A ideia geral das DC-CNNs é explorar o contexto da informação considerando sua direção causal. Em termos práticos, operações de convolução nestas CNNs são restritas a apenas uma direção. É mais fácil entender isso com um exemplo. Abaixo, temos o esquema de uma convolução causal 1-D de largura 2. Ou seja, cada neurônio observa duas entradas. Como a convolução é causal, note que o $t$-ésimo neurônio só pode observar entradas $t$ e $t-1$.

<img src="images/WaveNet_causalconv.png" alt="Convolução causal" style="width: 600px;"/>

Este tipo de operação permite a previsão de uma saída, pela observação de entradas anteriores a ela. Contudo, note no exemplo, que para observar 5 entradas, a rede precisa de 3 camadas ocultas. Para observar uma série com 16 entradas, seriam necessárias 14 camadas, tornando a rede rapidamente  inviável para sequencias maiores.

Uma forma de resolver este problema é atraves de _convoluções dilatadas_, ou seja convoluções aplicadas à entrada com intervalos de tamanhos pré-definidos. Por exemplo, com apenas 3 camadas é possível observar 16 entradas se cada camada possuir o dobro de intervalos da camada anterior, como visto abaixo:

<img src="images/WaveNet_dilatedconv.png" alt="Convolução causal" style="width: 600px;"/>

A quantidade de intervalos entre duas observações é chamada _fator de dilatação_. Uma convolução normal pode ser vista como uma convolução em que o fator de dilatação $k = 1$. Com $k=2$, a convolução saltaria uma a cada duas entradas. Com $k=4$, seriam saltadas 3 entradas. 

Estas operações de convolução possibilitam que se aumente o "campo receptivo" de um neurônio exponencialmente com um aumento apenas linear do número de parâmetros. Assim, é possível se explorar a sequência de elementos tanto de forma detalhada quanto geral, sem grande custo processamento.

Assim, usando convoluções causais dilatadas, é possível usar CNNs para preverem o próximo elemento de uma sequência, tornando possível o seu uso como previsores temporais, como ilustrado abaixo:

<img src="images/dilation_animation.gif" alt="Animação de dilations" style="width: 500px;"/>

Neste notebook, vamos implementar esta estratégia de previsão.

### Uma arquitetura CNN dilatada

Em particular, vamos implementar a rede _seriesnet_, uma DN-CNN com a arquitetura ilustrada a seguir:

<img src="images/dilatedCNN.png" alt="CNN dilatada" style="width: 500px;"/>

Nesta arquitetura, em particular, 7 camadas são empilhadas. Cada camada corresponde a um bloco com diferente fator de dilatação (bloco DC-CNN). A saída de todos os blocos é somada e transformada por uma operação _relu_. O resultado é então combinado em uma convolução de largura 1 que gera a previsão $\hat{y}$. 

Cada bloco DC-CNN corresponde a um conjunto de filtros de convolução causal dilatada, com saída tanto para o próximo bloco quanto para o somador final. Ambas as saídas ainda são transformadas por uma operação relu e combinadas via camadas de convolução 1x1. A saída para o próximo bloco é residual. A seguir, temos a implementação desta arquitetura em Keras.

In [ ]:
import tensorflow as tf

#Se vocês estiverem tendo crashes estranhos quando rodarem as convolucoes, 
#pode ser um problema do CUDA nao estar gerenciando bem a memoria. Descomentem essas linhas nesse caso
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

from keras.layers import Conv1D, Input, Add, Activation, Dropout
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers



In [ ]:
def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def block(block_input):        
        residual =    block_input
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False,
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(block_input)                    
        selu_out =    Activation('selu')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(selu_out)
        
        c1x1_out =    Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(selu_out)
                      
        block_out =   Add()([residual, c1x1_out])
        
        return block_out, skip_out
    return block

In [ ]:
def DC_CNN_Model(length):
    visible = Input(shape=(length, 1))
    
    l1a, l1b = DC_CNN_Block(32,2,1,0.001)(visible)    
    l2a, l2b = DC_CNN_Block(32,2,2,0.001)(l1a) 
    l3a, l3b = DC_CNN_Block(32,2,4,0.001)(l2a)
    l4a, l4b = DC_CNN_Block(32,2,8,0.001)(l3a)
    l5a, l5b = DC_CNN_Block(32,2,16,0.001)(l4a)
    l6a, l6b = DC_CNN_Block(32,2,32,0.001)(l5a)
    l7a, l7b = DC_CNN_Block(32,2,64,0.001)(l6a)

    l8 =   Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])  
    
    l9 =   Activation('relu')(l8)
           
    yhat =  Conv1D(1,1, activation='linear', use_bias=False, 
            kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=42),
            kernel_regularizer=l2(0.001))(l9)

    model = Model(inputs=visible, outputs=yhat)
    
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0)

    model.compile(loss='mae', optimizer=adam, metrics=['mse'])
    return model

### Usando CNNs dilatadas para previsão de séries de tempo

Para testar nosso modelo, vamos usar uma série com informaçoes sobre o número de passageiros em vôos nos EUA entre 1949 e 1961.

In [ ]:
import pandas as pd

In [ ]:
airline = pd.read_csv('data/airline.csv')

In [ ]:
airline['passengers'].plot()

Esta série apresenta clara sazonalidade e uma tendência de crescimento. A seguir, Vamos agora fazer uma previsão para um horizonte de 40 dias:

In [ ]:
forecast_horizon = 40

In [ ]:
Xpast = airline['passengers'].iloc[:-forecast_horizon]
Xreal = airline['passengers'].iloc[-forecast_horizon:] 

In [ ]:
Xpast = np.atleast_2d(np.asarray(Xpast)).T

In [ ]:
length = len(Xpast)-1

In [ ]:
model = DC_CNN_Model(length)

In [ ]:
X = Xpast[:-1].reshape(1,length,1)
y = Xpast[1:].reshape(1,length,1)    

In [ ]:
print(X[0,:5,:].T)
print(y[0,:5,:].T)

In [ ]:
model.fit(X, y, epochs=500)

In [ ]:
#model.summary()
X_test_initial = Xpast[1:].reshape(1,length,1)
X_trfit = model.predict(X_test_initial)

In [ ]:
plt.plot(Xpast, label = 'past')
plt.plot(X_trfit.flatten(), label = 'fit')
plt.legend(loc = 'upper left')

In [ ]:
preds = np.zeros(forecast_horizon)
preds[0] = model.predict(X_test_initial)[0,-1,0]

for i in range(1, forecast_horizon):
    X_test_i = np.append(X_test_initial[0,i:,:], preds[:i])
    preds[i] = model.predict(X_test_i.reshape(1, length, 1))[0,-1,0]

In [ ]:
plt.plot(Xreal, label = 'real')
plt.plot(pd.DataFrame(preds, index=Xreal.index), label = 'pred')
plt.legend()

Juntando tudo:

In [ ]:
def evaluate_timeseries(timeseries, predict_size, n_epochs = 3000):
    # timeseries input is 1-D numpy array
    # forecast_size is the forecast horizon
    
    # remove missing data
    timeseries = timeseries[~pd.isnull(timeseries)]

    length = len(timeseries)-1

    timeseries = np.atleast_2d(np.asarray(timeseries))
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T 

    model = DC_CNN_Model(length)
    print('\n\nModel input size',model.input_shape, 'output size' , model.output_shape)
    
    model.summary()

    # prime model
    X = timeseries[:-1].reshape(1,length,1)
    y = timeseries[1:].reshape(1,length,1)    
    model.fit(X, y, epochs=n_epochs)
    
    # get fit for training data
    X_test_initial = timeseries[1:].reshape(1,length,1)
    X_trfit = model.predict(X_test_initial) 
        
    # forecast is created by predicting next future 
    # value based on previous predictions
    preds = np.zeros(forecast_horizon)
    preds[0] = model.predict(X_test_initial)[0,-1,0]
    for i in range(1, forecast_horizon):
        X_test_i = np.append(X_test_initial[0,i:,:], preds[:i])
        preds[i] = model.predict(X_test_i.reshape(1, length, 1))[0,-1,0]
        
    return preds, X_trfit.flatten()

In [ ]:
Xpred, Xfit = evaluate_timeseries(Xpast, forecast_horizon, n_epochs = 4000)

Finalmente, podemos observar o desempenho do modelo como previsor. Note que este resultado foi obtido ao se prever o próximo mês com base na previsão feita para o mês anterior.

In [ ]:
plt.plot(Xpast, label = 'past')
plt.plot(Xfit, label = 'fit')
plt.plot(Xreal, label = 'real')
plt.plot(pd.DataFrame(Xpred, index=Xreal.index), label = 'pred')
plt.legend(loc = 'upper left')

O modelo foi capaz de capturar tanto a sazonalidade quanto a tendência, embora o erro acumulado seja muito alto para o horizonte observado.

__Exercício__: qual o efeito de adicionar dropout às camadas mais prodfundas do modelo. Em particular, avalie o modelo considerando dropout de 20% nas últimas duas camadas (as duas que enxergam mais no passado).

Esta aula foi baseada no material de Krist Papadopoulos, disponível em https://github.com/kristpapadopoulos/seriesnet. Também foi usado material da DeepMind, os criadores da Wavenet, disponível no blog https://deepmind.com/blog/wavenet-generative-model-raw-audio/ e de Joseph Eddy, disponível em https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/.